In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException 
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

import time
import re
from urllib.parse import urlparse, parse_qs

from shapely.geometry import Point, Polygon

from blueribbon_crawling import BlueRibbonCrawler
from instagram_crawling import InstagramCrawler
from load_seoul_hotspot import load_seoul_hotspots

In [7]:
class DatePopCrawler:
    def __init__(self, location, keyword, hotspots):
        self.location = location
        self.keyword = keyword
        self.search_word = location + " " + keyword
        self.hotspots = hotspots

        self.is_food = False

        self.data = pd.DataFrame(columns=['name', 'category', 'is_food', 'instagram_link', 'instagram_post', 'instagram_follower', 'visitor_review_count', 
                           'blog_review_count', 'distance_from_subway', 'on_tv','parking_available' , 'no_kids', 'pet_available', 'seoul_michelin',
                           'age-2030', 'gender-balance', 'on_blue_ribbon', 'image_urls'])
        self.empty_searchIframe = """//*[@id="_pcmap_list_scroll_container"]"""
        self.empty_entryIframe = """//*[@id="app-root"]"""
        self.empty_root = """//*[@id="root"]"""

        self.search_iframe = """//*[@id="searchIframe"]"""
        self.entry_iframe = """//*[@id="entryIframe"]"""

        self.store_dict = {
            'name': "",
            'category': "",
            'is_food': False,
            "instagram_link": None,
            "instagram_post": None,
            "instagram_follower": None,
            "visitor_review_count": 0,
            "blog_review_count": 0,
            "distance_from_subway": None,
            "on_tv": False,
            "parking_available": False,
            "no_kids": False,
            "pet_available": False,
            "seoul_michelin": False,
            "age-2030" : None,
            "gender-balance": None,
            "on_blue_ribbon": None,
            "image_urls": [],
        }

        if keyword == "맛집":
            self.blue_ribbon_crawler = BlueRibbonCrawler(self.location)
            self.is_food = True
            self.blue_ribbon_crawler.crawling()

        self.driver = self.initialize_driver()
        self.wait = WebDriverWait(self.driver, 10)

    def initialize_driver(self):
        options = webdriver.ChromeOptions()
        options.add_argument("--enable-logging")
        options.add_argument("--v=1")  # 로그 레벨 설정

        driver = webdriver.Chrome(options=options)
        driver.get("https://map.naver.com/")

        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[0])

        return driver
    
    def initialize_dictionary(self):
        self.store_dict = {
            "name": "",
            "category": "",
            "is_food": self.is_food,
            "instagram_link": None,
            "instagram_post": None,
            "instagram_follower": None,
            "hot_spot" : False,
            "visitor_review_count": 0,
            "blog_review_count": 0,
            "distance_from_subway": None,
            "on_tv": False,
            "parking_available": False,
            "no_kids": False,
            "pet_available": False,
            "seoul_michelin": False,
            "age-2030" : None,
            "gender-balance": None,
            "on_blue_ribbon": None,
            "image_urls": [],
        }
    
    def search_keyword(self):
        self.wait.until(EC.presence_of_element_located((By.XPATH, self.empty_root)))

        css_selector = ".input_search"
        elem = self.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))

        elem.send_keys(self.search_word)
        time.sleep(1)
        elem.send_keys(Keys.RETURN)

    def get_into_store(self, i):
        # 크롤링 값을 저장할 dictionay 변수 초기화
        self.initialize_dictionary()

        self.driver.switch_to.default_content()
        self.wait.until(EC.presence_of_element_located((By.XPATH, self.empty_root)))
        self.wait.until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, self.search_iframe)))
        self.wait.until(EC.presence_of_element_located((By.XPATH, self.empty_searchIframe)))

        store_xpath = f"""//*[@id="_pcmap_list_scroll_container"]/ul/li[{i}]/div[1]/a[1]"""
        elem = self.wait.until(EC.element_to_be_clickable((By.XPATH, store_xpath)))
        time.sleep(2)
  
        self.driver.execute_script("arguments[0].scrollIntoView(true);", elem)
        self.driver.execute_script("arguments[0].click()", elem)

        time.sleep(1)

        self.driver.switch_to.default_content()
        self.wait.until(EC.presence_of_element_located((By.XPATH, self.empty_root)))
        self.driver.find_element(By.XPATH, self.empty_root)

        self.wait.until(EC.presence_of_element_located((By.XPATH, self.entry_iframe)))
        iframe_element = self.driver.find_element(By.XPATH, self.entry_iframe)
        iframe_src = iframe_element.get_attribute('src')
        self.driver.switch_to.frame(iframe_element)
        # self.wait.until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, self.entry_iframe)))
        self.driver.find_element(By.XPATH, self.empty_entryIframe)

        parsed_url = urlparse(iframe_src)
        query_params = parse_qs(parsed_url.query)
        lat = query_params.get('x')[0]
        lon = query_params.get('y')[0]
        print(lat+ ", " +lon)
        store_point = Point(lat, lon)


        print("핫스팟 판별")
        for i in range(len(self.hotspots)):
            print(f"{i}번째 지역")
            polygon = self.hotspots[i]["polygon_area"]
            if polygon.contains(store_point):
                self.store_dict["hot_spot"] = True # default = False
                break

        # 매장 클릭 시 "요청하신 페이지를 찾을 수 없습니다"라는 메시지를 갖는 에러 가끔 발생
        # "새로고침" 버튼 클릭하여 매장 정보 다시 불러오기
        try:
            WebDriverWait(self.driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[contains(text(), '요청하신 페이지를 찾을 수 없습니다.')]")))
            reset_xpath = """//a[contains(text(), "새로고침)]"""

            reset_elem = self.wait.until(EC.presence_of_element_located((By.XPATH, reset_xpath)))
            self.driver.execute_script("arguments[0].click()", reset_elem)
        except:
            pass

    def get_store_details(self):
        
        time.sleep(3)

        # 매장 이름, 카테고리
        try:
            store_name_xpath = """//*[@id="_title"]/div/span"""
            elem = self.wait.until(EC.presence_of_all_elements_located((By.XPATH, store_name_xpath)))

            self.store_dict['name'] = elem[0].text
            self.store_dict['category'] = elem[1].text
        except Exception as e:
            print("매장 이름, 카테고리 에러: ", e)
            # 희귀하게 매장 이름을 크롤링하지 못하는 에러 발생
            # 해당 매장 생략하고 다음 매장 크롤링 진행
            return False

        # 방문자 리뷰, 블로그 리뷰 개수
        try:
            elem_visitor = self.driver.find_element(By.XPATH, value="//a[contains(text(), '방문자리뷰')]")
            elem_blog = self.driver.find_element(By.XPATH, value="//a[contains(text(), '블로그리뷰')]")

            visitor_review_count = int(re.findall(r'\d+', elem_visitor.text.replace(",", ""))[0])
            blog_review_count = int(re.findall(r'\d+', elem_blog.text.replace(",", ""))[0])

            self.store_dict['visitor_review_count'] = visitor_review_count
            self.store_dict['blog_review_count'] = blog_review_count
        except NoSuchElementException:
            self.store_dict['visitor_review_count'] = None
            self.store_dict['blog_review_count'] = None

        # 인스타그램 계정 존재 확인
        try:
            elem = self.driver.find_element(By.XPATH, value= "//a[contains(text(), '인스타그램')]")
            instagram_url = elem.get_attribute('href')

            # 인스타그램 계정 url 뒤에 queryParameter가 붙은 경우 or "/"가 하나 더 붙은 경우
            # 두 경우 모두, 해당 url에 "/embed"를 concatenation하면 응답으로 "알 수 없는 페이지"
            # 때문에 url에서 해당 인스타그램 계정의 이름에 해당하는 부분 이후는 제거
            if instagram_url == "https://instagram.com" or instagram_url == "https://instagram.com/" or instagram_url == "http://instagram.com" or instagram_url == "http://instagram.com/":      
                self.store_dict['instagram_link'] = None
            else:
                if "?" in instagram_url:
                    instagram_url = instagram_url.split("?")[0]

                if instagram_url.count("/") >= 4:
                    instagram_url = "/".join(instagram_url.split("/", 4)[:4])

                self.store_dict['instagram_link'] = instagram_url
        except NoSuchElementException:
            self.store_dict['instagram_link'] = None
            self.store_dict['instagram_post'] = None
            self.store_dict['instagram_follower'] = None
        except Exception as e:
            print("인스타그램 에러:", e)
            self.store_dict['instagram_link'] = None
            self.store_dict['instagram_post'] = None
            self.store_dict['instagram_follower'] = None

        # 서울 미쉐린 가이드
        try:
            michelin_xpath = """//div[a[contains(text(), '미쉐린 가이드 서울')]]"""
            self.driver.find_element(By.XPATH, michelin_xpath)
            self.store_dict['seoul_michelin'] = True
        except NoSuchElementException:
            self.store_dict['seoul_michelin'] = False
        except Exception as e:
            print("서울 미쉐린 가이드 에러:", e)
            self.store_dict['seoul_michelin'] = None
            
        # 지하철역 출구로부터 거리
        try:
            subway_xpath = "/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[1]/div/div"
            elem = self.driver.find_element(By.XPATH, subway_xpath)
            text = elem.text

            numbers = re.findall(r'\d+', text)
            if numbers:
                self.store_dict["distance_from_subway"] = numbers[-1]
        except NoSuchElementException:
            self.store_dict["distance_from_subway"] = None
        except Exception as e:
            print("지하철역 에러: ", e)
            self.store_dict["distance_from_subway"] = None

        # 방송 출연 여부
        try:
            tv_xpath =  """//strong[descendant::span[text()='TV방송정보']]"""
            self.driver.find_element(By.XPATH, tv_xpath)
            self.store_dict['on_tv'] = True
        except NoSuchElementException:
            self.store_dict['on_tv'] = False
        except Exception as e:
            print("방송 출연 에러: ", e)
            self.store_dict['on_tv'] = None

        # 주차 가능, 반려동물 동반, 노키즈존
        try:
            convenient_xpath = "//strong[descendant::span[text()='편의']]/ancestor::div[1]/div/div"
            elem = self.driver.find_element(By.XPATH, convenient_xpath)
            convenients = elem.text

            for parking in ["주차", "발렛파킹"]:
                if parking in convenients:
                    self.store_dict["parking_available"] = True
                    break

            if "반려동물 동반" in convenients:
                self.store_dict["pet_available"] = True

            if "노키즈존" in convenients:
                self.store_dict["no_kids"] = True
        except NoSuchElementException:
            self.store_dict["parking_available"] = False
            self.store_dict["no_kids"] = False
            self.store_dict["pet_available"] = False
        except Exception as e:
            print("주차, 반려동물, 노키즈 에러: ", e)
            self.store_dict["parking_available"] = False
            self.store_dict["no_kids"] = False
            self.store_dict["pet_available"] = False

        # DataLab: 연령별 / 성별 검색 인기도
        try:
            # entryIframe 스크롤 끝까지 내려서 모든 컨텐츠 로딩
            last_height = self.driver.execute_script("return document.body.scrollHeight")
            while True:
                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(0.5)
                new_height = self.driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # DataLab 항목 찾고 해당 element로 스크롤 이동
            datalab_xpath = """//div[h2/span[contains(text(), '데이터랩')]]"""
            datalab_elem = self.driver.find_element(By.XPATH, datalab_xpath)
            self.driver.execute_script("arguments[0].scrollIntoView(true);", datalab_elem)

            # "테마키워드"라는 text가 있는 경우, "더보기 버튼을 눌러줘야 연령별/성별 검색어 비율 확인 가능
            try:
                theme_keyword_xpath = """.//div/div/div/h3[contains(text(), '테마키워드')]"""
                datalab_elem.find_element(By.XPATH, theme_keyword_xpath)
                button_elem = datalab_elem.find_element(By.XPATH, ".//div[2]/div/a")
                self.driver.execute_script("arguments[0].click()", button_elem)
            except NoSuchElementException:
                pass
            except Exception as e:
                print("데이터랩 더보기 버튼 에러: ", e)
            
            # 20대와 30대가 top 1, 2를 차지하는지 확인
            age_xpath = """//*[@id="bar_chart_container"]/ul/li/div[1]/span/span[1]"""
            age_elements = self.wait.until(EC.presence_of_all_elements_located((By.XPATH, age_xpath)))
            percentage_by_age = [round(float(item.text.replace('%', '')), 2) for item in age_elements]
            top_two = sorted(percentage_by_age, reverse=True)[:2]
            is_2030_in_top_two = percentage_by_age[1] in top_two and percentage_by_age[2] in top_two
            if is_2030_in_top_two:
                self.store_dict["age-2030"] = True
            else:
                self.store_dict["age-2030"] = False

            # 남성의 비율이 50%를 넘는지 확인
            gender_xpath = """//*[@id="pie_chart_container"]/div/*[local-name()='svg']/*[local-name()='g'][1]/*[local-name()='g'][3]/*[local-name()='g'][4]/*[local-name()='g']/*[local-name()='text'][2]"""
            gender_elements = self.wait.until(EC.presence_of_all_elements_located((By.XPATH, gender_xpath)))
            female, male = [round(float(item.text.replace("%", "")), 0) for item in gender_elements]
            if male > 50:
                self.store_dict["gender-balance"] = False
            else:
                self.store_dict["gender-balance"] = True
        except NoSuchElementException:
            self.store_dict["age-2030"] = None
            self.store_dict["gender-balance"] = None
        except Exception as e:
            print("데이터랩 에러: ", e)
            self.store_dict["age-2030"] = None
            self.store_dict["gender-balance"] = None

        # 블루 리본 등재 여부 -> "맛집" 키워드일 경우에만 확인
        if self.is_food == True:
            if self.store_dict["name"] in self.blue_ribbon_crawler.data["name"].values:
                self.store_dict["on_blue_ribbon"] = True
            else:
                self.store_dict["on_blue_ribbon"] = False

        # 대표사진 크롤링
        try:
            imgtab_xpath = "//a[.//span[contains(text(),'사진')]]"
            elem = self.driver.find_element(By.XPATH, imgtab_xpath)
            self.driver.execute_script("arguments[0].click()", elem)
            time.sleep(2) # 사진 로딩 대기

            images_xpath = """/html/body/div[3]/div/div/div/div[6]/div[4]/div/div/div/div/a/img"""
            self.wait.until(EC.presence_of_all_elements_located((By.XPATH, images_xpath)))

            image_elements = self.wait.until(
                EC.presence_of_all_elements_located((By.XPATH, images_xpath))
            )
            image_urls = [img.get_attribute('src') for img in image_elements][:10] # 최대 10개의 이미지 url
            self.store_dict["image_urls"] = image_urls
        except NoSuchElementException:
            self.store_dict["image_urls"] = None
        except TimeoutException:
            self.store_dict["image_urls"] = None
        # except Exception as e:
        #     print("대표사진 에러: ", e)
        #     self.store_dict["image_urls"] = None

        # 인스타그램 크롤링
        if self.store_dict['instagram_link'] != None: # 인스타그램 계정이 있는 경우에만 실행
            try:
                instagram_embed_url = self.store_dict['instagram_link'] + "/embed"

                # 인스타그랩 탭으로 이동
                self.driver.switch_to.window(self.driver.window_handles[1])
                self.driver.get(instagram_embed_url)

                xpath = """//span[contains(., '팔로워') and contains(., '게시물')]/span/span"""
                elements = self.wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))

                follower = elements[0].text
                post = elements[1].text
                self.store_dict["instagram_follower"] = follower
                self.store_dict["instagram_post"] = post
            except NoSuchElementException:
                self.store_dict['instagram_link'] = None
                self.store_dict["instagram_follower"] = None
                self.store_dict["instagram_post"] = None
            except Exception as e:
                print("인스타그램 에러: ", e)
                self.store_dict['instagram_link'] = None
                self.store_dict["instagram_follower"] = None
                self.store_dict["instagram_post"] = None

            # 네이버지도 탭으로 복귀
            self.driver.switch_to.window(self.driver.window_handles[0])
        
        print(self.store_dict)

        return True

    # 한 매장에 대한 크롤링 진행 후, 해당 정보를 DataFrame에 Insertion
    def insert_into_dataframe(self):
        new_data = pd.DataFrame([self.store_dict])
        self.data = pd.concat([self.data, new_data], ignore_index=True)

    # 한 페이지 크롤링
    # 페이지별로 전체 매장 개수가 다름
    # 첫 페이지의 경우 54개, 그 이후에는 50개
    # 예외 케이스 있을 수 있기 때문에 유연하게 대응할 수 있도록 update 필요
    def crawling_one_page(self):
        self.driver.switch_to.default_content()
        self.wait.until(EC.presence_of_element_located((By.XPATH, self.empty_root)))
        self.wait.until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, self.search_iframe)))
        self.wait.until(EC.presence_of_element_located((By.XPATH, self.empty_searchIframe)))

        li_xpath = """//*[@id="_pcmap_list_scroll_container"]/ul/li"""
        store_elements = self.wait.until(EC.presence_of_all_elements_located((By.XPATH, li_xpath)))
        store_count = len(store_elements)
        self.driver.execute_script("arguments[0].scrollIntoView(true);", store_elements[-1])
        while True:
            time.sleep(0.5)
            store_elements = self.wait.until(EC.presence_of_all_elements_located((By.XPATH, li_xpath)))
            new_store_count = len(store_elements)

            if store_count == new_store_count:
                break
            store_count = new_store_count
            self.driver.execute_script("arguments[0].scrollIntoView(true);", store_elements[-1])
            
        for i in range(1, store_count + 1):
            print("="*3+f"{i} 번째 매장"+ "="*3)
            self.get_into_store(i=i)
            if self.get_store_details():
                self.insert_into_dataframe()

        

    # 다음 페이지로 이동
    def move_to_next_page(self):
        self.driver.switch_to.default_content()
        self.wait.until(EC.presence_of_element_located((By.XPATH, self.empty_root)))
        self.wait.until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, self.search_iframe)))

        nextpage_xpath = """//a[span[contains(text(),'다음페이지')]]"""
        next_page_button = self.wait.until(EC.element_to_be_clickable((By.XPATH, nextpage_xpath)))
        next_page_button.click()

        time.sleep(2)

In [8]:
if __name__ == "__main__":
    
    location = "가로수길"
    keyword = "맛집"
    search_word = location + " " + keyword
    seoul_hotspots = load_seoul_hotspots()
    crawler = DatePopCrawler(location=location, keyword= keyword, hotspots=seoul_hotspots)

    crawler.search_keyword()
    for page in range(1, 7):
        print("="*10+f"page {page}"+ "="*10)
        crawler.crawling_one_page()
        time.sleep(1)
        crawler.move_to_next_page()
        print(crawler.data)
    crawler.driver.quit()
    print(f"Crawling for {location} {keyword} is done.")
    

Blue Ribbon Survey crawling start!
Crawling page 0......
Crawling page 1......
Crawling page 2......
Crawling page 3......
Crawling page 4......
Crawling page 5......
Crawling page 6......
Crawling page 7......
Blue Ribbon Survey crawling done!
==========page 1==========
===1 번째 매장===
127.030186593093, 37.5193587335594
핫스팟 판별
0번째 지역
1번째 지역
2번째 지역
3번째 지역
4번째 지역
5번째 지역
6번째 지역
7번째 지역
8번째 지역
9번째 지역
10번째 지역
11번째 지역
12번째 지역
13번째 지역
0
3
{'name': '에이뿔램 논현점', 'category': '양갈비', 'is_food': True, 'instagram_link': None, 'instagram_post': None, 'instagram_follower': None, 'hot_spot': False, 'visitor_review_count': 278, 'blog_review_count': 697, 'distance_from_subway': '694', 'on_tv': False, 'parking_available': True, 'no_kids': False, 'pet_available': False, 'seoul_michelin': False, 'age-2030': True, 'gender-balance': False, 'on_blue_ribbon': False, 'image_urls': ['https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20231010_4%2F1696904174

KeyboardInterrupt: 

In [ ]:
seoul_hotspots = [
    {
    "location" : "강남역1",
    "polygon_area" : Polygon([
        (127.021136, 37.503349),
        (127.026467, 37.505041),
        (127.027580, 37.502774),
        (127.028410, 37.502268),
        (127.029303, 37.502572),
        (127.033448, 37.494069),
        (127.026524, 37.491984),
    ])},
    {
    "location" : "강남역2",
    "polygon_area" : Polygon([
        (127.017214, 37.507406),
        (127.019084, 37.508937),
        (127.019707, 37.509196),
        (127.019811, 37.508949),
        (127.024665, 37.510444),
        (127.026951, 37.508937),
        (127.027159, 37.506865),
        (127.027797, 37.505499),
        (127.019544, 37.503097),
        (127.019128, 37.504109),
        (127.018446, 37.503968),
    ])},
    {
    "location" : "건대",
    "polygon_area" : Polygon([
        (127.066675, 37.546900),
        (127.069465, 37.546049),
        (127.070216, 37.546814),
        (127.071911, 37.546746),
        (127.073434, 37.545896),
        (127.074121, 37.547257),
        (127.079155, 37.545406),
        (127.085396, 37.545100),
        (127.083222, 37.539943),
        (127.083738, 37.536145),
        (127.078150, 37.534610),
        (127.078021, 37.534283),
        (127.077709, 37.534100),
        (127.076044, 37.834406),
        (127.076132, 37.534787),
        (127.075326, 37.534943),
        (127.075489, 37.535373),
        (127.069490, 37.537482),
        (127.069659, 37.537874),
        (127.062827, 37.540169),
    ])},
    {
    "location" : "광화문",
    "polygon_area" : Polygon([
        (126.971937, 37.570065),
        (126.968866, 37.568179),
        (126.967752, 37.569169),
        (126.966655, 37.569370),
        (126.965896, 37.570025),
        (126.964428, 37.572606),
        (126.965660, 37.573904),
        (126.967398, 37.574492),
        (126.968849, 37.575629),
        (126.969271, 37.577194),
        (126.972258, 37.577154),
        (126.972443, 37.576605),
        (126.974207, 37.576465),
        (126.973635, 37.583502),
        (126.977027, 37.583836),
        (126.979234, 37.583448),
        (126.980259, 37.582636),
        (126.979644, 37.577902),
        (126.980611, 37.577649),
        (126.980930, 37.578308),
        (126.981465, 37.578317),
        (126.981613, 37.578605),
        (126.982163, 37.578650),
        (126.983027, 37.575552),
        (126.982979, 37.570227),
    ])},
    {
    "location" : "대학로",
    "polygon_area" : Polygon([
        (126.997377, 37.575873),
        (126.996574, 37.581103),
        (126.995276, 37.584869),
        (126.993634, 37.587233),
        (126.989622, 37.587088),
        (126.989386, 37.589188),
        (126.993422, 37.590125),
        (126.994559, 37.589198),
        (126.995368, 37.588484),
        (126.996630, 37.587763),
        (126.997355, 37.589272),
        (126.998566, 37.589960),
        (126.999218, 37.589700),
        (127.000068, 37.586847),
        (127.001043, 37.585741),
        (127.001432, 37.585819),
        (127.003661, 37.587571),
        (127.005749, 37.586908),
        (127.007359, 37.583050),
        (127.008684, 37.577243),
        (127.006581, 37.576581),
        (127.004988, 37.575627),
        (127.002234, 37.576177),
    ])},
    {
    "location" : "명동",
    "polygon_area" : Polygon([
        (126.987548, 37.566779),
        (126.981498, 37.566710),
        (126.979189, 37.566953),
        (126.978546, 37.564938),
        (126.979349, 37.563941),
        (126.978765, 37.563779),
        (126.979028, 37.562482),
        (126.979028, 37.561289),
        (126.980255, 37.561393),
        (126.979963, 37.559875),
        (126.980299, 37.559597),
        (126.981176, 37.559852),
        (126.981293, 37.558555),
        (126.982725, 37.558902),
        (126.983260, 37.556512),
        (126.984246, 37.556856),
        (126.984574, 37.556311),
        (126.986103, 37.556962),
        (126.986935, 37.556043),
        (126.988503, 37.556161),
        (126.988699, 37.557312),
        (126.989476, 37.557510),
        (126.990068, 37.559966),
        (126.988912, 37.561175),
        (126.988690, 37.562994),
        (126.987617, 37.564643),
    ])},
    {
    "location" : "삼청동",
    "polygon_area" : Polygon([
        (126.989257, 37.577364),
        (126.986981, 37.577249),
        (126.984268, 37.576022),
        (126.983340, 37.578512),
        (126.981555, 37.578605),
        (126.981413, 37.578314),
        (126.980929, 37.578307),
        (126.980620, 37.577632),
        (126.979485, 37.577917),
        (126.980278, 37.581977),
        (126.981090, 37.582788),
        (126.981544, 37.583940),
        (126.980980, 37.588198),
        (126.982595, 37.588965),
        (126.986684, 37.588998),
        (126.988056, 37.588536),
        (126.988306, 37.587493),
        (126.987459, 37.586796),
        (126.987679, 37.583820),
        (126.989113, 37.584049),
        (126.989428, 37.583662),
        (126.989411, 37.582867),
        (126.989785, 37.582629),
        (126.989502, 37.581736),
    ])},
    {
    "location" : "가로수길",
    "polygon_area" : Polygon([
        (127.024016, 37.524918),
        (127.028160, 37.519681),
        (127.028428, 37.518877),
        (127.026505, 37.518026),
        (127.026818, 37.517515),
        (127.024493, 37.515662),
        (127.023584, 37.515532),
        (127.021572, 37.514894),
        (127.021021, 37.515969),
        (127.019828, 37.515532),
        (127.018502, 37.520414),
        (127.019575, 37.522726),
    ])},
    {
    "location" : "신촌(이대)",
    "polygon_area" : Polygon([
        (126.934654, 37.559554),
        (126.934487, 37.559012),
        (126.933625, 37.558620),
        (126.932918, 37.557277),
        (126.932602, 37.554812),
        (126.934447, 37.553040),
        (126.936337, 37.554161),
        (126.936975, 37.554247),
        (126.937199, 37.553122),
        (126.938682, 37.553847),
        (126.942353, 37.555036),
        (126.942940, 37.555664),
        (126.943112, 37.554712),
        (126.944284, 37.554626),
        (126.945899, 37.554293),
        (126.947513, 37.554320),
        (126.948352, 37.554644),
        (126.948811, 37.555331),
        (126.948503, 37.555787),
        (126.948736, 37.557629),
        (126.947944, 37.558759),
        (126.947756, 37.560176),
    ])},
    {
    "location" : "이태원(경리단길, 해방촌)",
    "polygon_area" : Polygon([
        (126.987377, 37.534316),
        (126.986263, 37.541223),
        (126.986280, 37.541997),
        (126.986015, 37.542248),
        (126.986434, 37.542656),
        (126.985570, 37.543418),
        (126.985790, 37.543694),
        (126.985833, 37.544201),
        (126.985166, 37.544681),
        (126.985249, 37.546559),
        (126.987505, 37.546001),
        (126.989017, 37.546243),
        (126.991016, 37.544420),
        (127.000816, 37.542928),
        (127.001995, 37.541572),
        (127.003629, 37.540798),
        (127.004719, 37.537420),
        (127.007648, 37.533291),
        (127.004717, 37.533366),
        (127.004668, 37.532984),
        (127.003299, 37.532486),
        (127.001072, 37.532719),
        (127.000638, 37.534141),
        (127.000430, 37.534351),
        (126.999054, 37.533336),
        (126.999556, 37.534364),
        (126.997757, 37.534302),
        (126.998108, 37.533384),
        (126.997903, 37.533011),
        (126.997349, 37.533115),
        (126.997026, 37.531911),
        (126.996616, 37.531497),
        (126.996754, 37.530361),
        (126.994989, 37.531521),
        (126.993803, 37.531439),
        (126.992014, 37.529036),
    ])},
    {
    "location" : "인사동",
    "polygon_area" : Polygon([
        (126.982872, 37.575507),
        (126.983036, 37.571046),
        (126.984301, 37.571027),
        (126.983973, 37.570252),
        (126.987810, 37.570304),
        (126.987967, 37.572969),
        (126.988418, 37.573082),
        (126.987657, 37.574328),
        (126.987327, 37.574223),
        (126.986790, 37.575544),
        (126.987671, 37.575802),
        (126.987178, 37.577284),
    ])},
    {
    "location" : "합정(상수)",
    "polygon_area" : Polygon([
        (126.904227, 37.550260),
        (126.905051, 37.550914),
        (126.906103, 37.551136),
        (126.905538, 37.551611),
        (126.906928, 37.551803),
        (126.908283, 37.552535),
        (126.908776, 37.552898),
        (126.909709, 37.552877),
        (126.909413, 37.553495),
        (126.911691, 37.554199),
        (126.915037, 37.555771),
        (126.916122, 37.554427),
        (126.920401, 37.550800),
        (126.923083, 37.550679),
        (126.922894, 37.545004),
        (126.911844, 37.545715),
    ])},
    {
    "location" : "홍대(연남)",
    "polygon_area" : Polygon([
        (126.915090, 37.555769),
        (126.918735, 37.557521),
        (126.916305, 37.560294),
        (126.924204, 37.564833),
        (126.927407, 37.562104),
        (126.928264, 37.563319),
        (126.934350, 37.559467),
        (126.934357, 37.559240),
        (126.933026, 37.558495),
        (126.931904, 37.557019),
        (126.930134, 37.554772),
        (126.929942, 37.554254),
        (126.929749, 37.553792),
        (126.931407, 37.552078),
        (126.927566, 37.550040),
        (126.924124, 37.550125),
        (126.923056, 37.550680),
        (126.920390, 37.550777),
        (126.916057, 37.554468),    
    ])},
    {
    "location" : "성수",
    "polygon_area" : Polygon([
        (127.035605, 37.540301),
        (127.035951, 37.543962),
        (127.033092, 37.544959),
        (127.040446, 37.550735),
        (127.044001, 37.551647),
        (127.053518, 37.550569),
        (127.060629, 37.548227),
        (127.067610, 37.548372),
        (127.059060, 37.533405),
    ])},
    ]